In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

In [1]:
import sys
!{sys.executable} -m pip install xgboost

In [9]:
df = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')

In [10]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### What don't I care about (shouldn't have much impact)
1. Passenger Name
2. Ticket
3. Cabin number
### What I might care about (already categorical or numerical data)
1. SibSp
2. Parch
3. Fare
4. Embarked
### What I definitely care about
1. Pclass
2. Sex
3. Age

### To do
1. Drop all the less useful information
2. Check for NA values in everything else, fill N/A with median 
3. Turn non numerical columns into numerical categories via OneHotEncoding
4. Split into train/test split and test a few models

In [11]:
df.drop(['Name','Ticket','Cabin'],axis=1,inplace=True)

In [12]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB


In [14]:
df['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [15]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

Because there are only two different values for Sex and 3 different values for Embarked, im not going to bother with OneHotEncoding and i'll just do it manually

In [16]:
num_sex = []
for i in df['Sex']:
    if i == 'female':
        num_sex.append(1)
    else:
        num_sex.append(0)        

In [17]:
df['Sex'] = num_sex

In [18]:
num_embarked = []
for i in df['Embarked']:
    if i == 'C':
        num_embarked.append(1)
    elif i == 'Q':
        num_embarked.append(2)
    else:
        num_embarked.append(0)
# This will also fill N/A values (2) with 0        

In [19]:
df['Embarked'] = num_embarked

In [20]:
df['Age'].value_counts()

24.00    30
22.00    27
18.00    26
19.00    25
30.00    25
         ..
55.50     1
70.50     1
66.00     1
23.50     1
0.42      1
Name: Age, Length: 88, dtype: int64

In [21]:
df['Age'].median()

28.0

In [22]:
df['Age'].fillna(df['Age'].median(),inplace=True)

In [23]:
# Now we have 0 missing values and all our data is in numerical form, we can attempt to build some models now
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 62.8 KB


In [24]:
from sklearn.model_selection import train_test_split
X = df.drop('Survived',axis=1)
y = df['Survived']

X_train,X_test,y_train,y_test = train_test_split(X,y)

In [25]:
len(X),len(y),len(X_train),len(X_test),len(y_train),len(y_test)

(891, 891, 668, 223, 668, 223)

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
np.random.seed(42)
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [72]:
rfc.score(X_test,y_test)

0.8026905829596412

so our baseline score is 82% accurate, which is pretty decent, but lets ty our some other tests before we adjust hyperparams


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier

In [46]:
def testing_score(X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test):
    testing_dictionary = {
        'Random Forest' : RandomForestClassifier(),
        'Logistic Regression' : LogisticRegression(max_iter=750),
        'Extreme Gradient' : xgb.XGBRegressor(),
        'Kneighbors': KNeighborsClassifier()
    }
    np.random.seed(42)
    model_scores = {}
    for name,model in testing_dictionary.items():
        model.fit(X_train,y_train)
        model_scores[name] = model.score(X_test,y_test)
    return model_scores

In [47]:
scores = testing_score()
scores

{'Random Forest': 0.8026905829596412,
 'Logistic Regression': 0.7802690582959642,
 'Extreme Gradient': 0.2077519102041635,
 'Kneighbors': 0.57847533632287}

Oof Extreme Gradient and KNeighbors really didn't do too good, lets continue with hyperparameter adjustment with the other two


In [63]:
Logistic_grid = {'C':np.logspace(-4,4,20),
                'solver':['liblinear'],
                'penalty': ['l1','l2']}

Forest_grid = {'n_estimators': np.arange(100,250,50),
               'max_depth' : [None,3,5,10],
               'min_samples_split' : np.arange(2,20,2),
               'min_samples_leaf' : np.arange(1,20,2)
              }


In [52]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [57]:
# Logistic Regression Hyperparam Tuning
np.random.seed(42)
# Lets start with GridSearchCV
gs_log_reg = GridSearchCV(LogisticRegression(),
                          param_grid=Logistic_grid,
                          cv = 5)
# and fit the model
gs_log_reg.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                         'penalty': ['l1', 'l2'], 'solver': ['liblinear']})

In [58]:
gs_log_reg.best_params_

{'C': 0.08858667904100823, 'penalty': 'l2', 'solver': 'liblinear'}

In [59]:
gs_log_reg.score(X_test,y_test)

0.7892376681614349

In [69]:
# Random Forest HyperparamTuning
np.random.seed(42)
gs_rf = RandomizedSearchCV(RandomForestClassifier(),
                    param_distributions = Forest_grid,
                    cv=5,
                    n_iter = 50,
                    verbose=True)
gs_rf.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:  1.2min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=50,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([100, 150, 200])},
                   verbose=True)

In [70]:
gs_rf.best_params_

{'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 3,
 'max_depth': 10}

In [71]:
gs_rf.score(X_test,y_test)

0.8026905829596412